In [2]:
import pandas as pd
import matplotlib.pyplot as plt

In [4]:
df = pd.read_csv('data/traffic_collison.csv')
df.shape

(553780, 19)

In [5]:
df.head()

,X,Y,OBJECTID,EventUniqueId,OccurrenceDate,Month,Day_of_Week,Year,Hour,Division,Atom,Neighbourhood,Fatalities,Injury_Collisions,FTR_Collisions,PD_Collisions,Longitude,Latitude,ObjectId2
0,0.000000e+00,0.000000e+00,401,GO-20148000204,2014/01/03 05:00:00+00,January,Friday,2014,8,NSA,NSA,NSA,0,NO,NO,YES,0.000000,0.000000,1
1,-8.833635e+06,5.414591e+06,402,GO-20148000205,2014/01/03 05:00:00+00,January,Friday,2014,12,D54/D55,68,North Riverdale (68),0,NO,NO,YES,-79.353893,43.670491,2
2,0.000000e+00,0.000000e+00,403,GO-20148000206,2014/01/03 05:00:00+00,January,Friday,2014,8,NSA,NSA,NSA,0,NO,NO,YES,0.000000,0.000000,3
3,-8.838998e+06,5.413446e+06,404,GO-20148000208,2014/01/03 05:00:00+00,January,Friday,2014,11,D52,79,University (79),0,NO,NO,YES,-79.402070,43.663052,4
4,-8.840999e+06,5.434235e+06,405,GO-20148000209,2014/01/03 05:00:00+00,January,Friday,2014,13,D32,50,Newtonbrook East (50),0,NO,NO,YES,-79.420043,43.798000,5


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 553780 entries, 0 to 553779
Data columns (total 19 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   X                  553780 non-null  float64
 1   Y                  553780 non-null  float64
 2   OBJECTID           553780 non-null  int64  
 3   EventUniqueId      553780 non-null  object 
 4   OccurrenceDate     553780 non-null  object 
 5   Month              553780 non-null  object 
 6   Day_of_Week        553780 non-null  object 
 7   Year               553780 non-null  int64  
 8   Hour               553780 non-null  int64  
 9   Division           553780 non-null  object 
 10  Atom               553780 non-null  object 
 11  Neighbourhood      553780 non-null  object 
 12  Fatalities         553780 non-null  int64  
 13  Injury_Collisions  553780 non-null  object 
 14  FTR_Collisions     553780 non-null  object 
 15  PD_Collisions      553780 non-null  object 
 16  Lo

In [7]:
df.describe()

,X,Y,OBJECTID,Year,Hour,Fatalities,Longitude,Latitude,ObjectId2
count,5.537800e+05,5.537800e+05,553780.000000,553780.000000,553780.000000,553780.000000,553780.000000,553780.000000,553780.000000
mean,-7.589603e+06,4.655751e+06,276890.500000,2017.530443,13.466469,0.000916,-68.178567,37.540806,276890.500000
std,3.078091e+06,1.888224e+06,159862.660373,2.313957,4.933017,0.030835,27.650962,15.225305,159862.660373
min,-8.910563e+06,0.000000e+00,1.000000,2014.000000,0.000000,0.000000,-80.044952,0.000000,1.000000
25%,-8.844152e+06,5.411159e+06,138445.750000,2016.000000,10.000000,0.000000,-79.448368,43.648191,138445.750000
50%,-8.836113e+06,5.418853e+06,276890.500000,2018.000000,14.000000,0.000000,-79.376156,43.698178,276890.500000
75%,-8.824347e+06,5.427770e+06,415335.250000,2019.000000,17.000000,0.000000,-79.270453,43.756063,415335.250000
max,0.000000e+00,5.533134e+06,553780.000000,2022.000000,23.000000,3.000000,0.000000,44.435809,553780.000000
